In [1]:
import os
import cv2
import random
import shutil
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from PIL import Image
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
train_dir = 'data/Training'
test_dir = 'data/Testing'
classes = os.listdir(train_dir)

In [3]:
resize_size = 256
crop_size = 224

def preprocess_image(image):
    image = tf.image.resize(image, [resize_size, resize_size], method=tf.image.ResizeMethod.BILINEAR) #크기 조절
    image = tf.image.central_crop(image, central_fraction=crop_size / resize_size) #중앙 224x224
    image = tf.math.divide(image, 255.0) #normalize
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    image = (image - mean) / std #다 normalize

    return image

In [4]:
from sklearn.preprocessing import LabelEncoder

X = [] #Image
y = [] #class
for i in classes:
    for data_dir in [train_dir, test_dir]:
        folderPath = os.path.join(data_dir,i)
        for j in tqdm(os.listdir(folderPath)):
            img = cv2.imread(os.path.join(folderPath,j)) #이미지 읽기
            img = preprocess_image(img) #전처리
            X.append(img) #X list 넣고
            y.append(i) # y list
X = np.array(X)
y = np.array(y)
y = tf.keras.utils.to_categorical([classes.index(label) for label in y]) #문자열 -> [0,3]

  7%|██▋                                      | 26/395 [00:00<00:02, 144.70it/s]

Metal device set to: Apple M2 Pro


100%|██████████████████████████████████████████| 74/74 [00:00<00:00, 167.02it/s]


In [5]:
X_train, xx, y_train, yy = train_test_split(X,y, test_size=0.2, random_state=42) #training 분할 (train,val)
X_train, X_val, y_train, y_val = train_test_split(xx,yy, test_size=0.5, random_state=42) #training 분할 (train,val)

In [6]:
#그래프 그려주는 거 (성능 그래프)
def plot_acc_model(acc, val_acc, epochs):
    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend(loc=0)
    plt.figure()
    plt.show()
    
def plot_loss_model(loss, val_loss, epochs):
    plt.plot(epochs, loss, 'r', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend(loc=0)
    plt.figure()
    plt.show()

In [7]:
from tensorflow.keras.applications import VGG16, EfficientNetV2L, ResNet50

base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model_vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import os

epochs=50

version = "model_adding_ver1"
def create_model(base_model, output_size, model_name):
    model_dir = f"{version}/{model_name}/"
    for layer in base_model.layers:
        layer.trainable = False
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    output = Dense(output_size, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)
    best_model_checkpoint = ModelCheckpoint(f"{model_dir}best_model.h5", monitor='val_loss', save_best_only=True, mode='min', verbose=1)
    csv_logger = CSVLogger(f'{version}/training_log_{model_name}.csv', separator=',', append=False)
    return model, [early_stopping, best_model_checkpoint, csv_logger]


base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_vgg, callbacks_vgg = create_model(base_model_vgg, output_size=4, model_name='vgg')

base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_resnet, callbacks_resnet = create_model(base_model_resnet, output_size=4, model_name='resnet')

history_vgg = model_vgg.fit(X_train, y_train,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            verbose=1,
                            callbacks=callbacks_vgg)

history_resnet = model_resnet.fit(X_train, y_train,
                                  epochs=epochs,
                                  validation_data=(X_val, y_val),
                                  verbose=1,
                                  callbacks=callbacks_resnet)


from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

for model_name in ["resnet", "vgg"]:
    best_model = keras.models.load_model(version+"/"+model_name+"/best_model.h5")
    y_pred = best_model.predict(X_test)
    y_pred_single_label = np.argmax(y_pred, axis=1)
    y_test_single_label = np.argmax(y_test, axis=1)
    precision = precision_score(y_test_single_label, y_pred_single_label, average='macro')
    recall = recall_score(y_test_single_label, y_pred_single_label, average='macro')
    f1 = f1_score(y_test_single_label, y_pred_single_label, average='macro')
    accuracy = accuracy_score(y_test_single_label, y_pred_single_label)
    precision = round(precision, 3)
    recall = round(recall, 3)
    f1 = round(f1, 3)
    accuracy = round(accuracy, 3)
    print(str(model_name))
    print(precision, recall, f1, accuracy)

Epoch 1/50


2023-11-19 01:47:18.784731: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


83/83 [==============================] - ETA: 0s - loss: 1.0109 - accuracy: 0.5793
Epoch 1: val_loss improved from inf to 0.77192, saving model to model_adding_ver1/vgg/best_model.h5
83/83 [==============================] - 25s 285ms/step - loss: 1.0109 - accuracy: 0.5793 - val_loss: 0.7719 - val_accuracy: 0.7279
Epoch 2/50
83/83 [==============================] - ETA: 0s - loss: 0.6664 - accuracy: 0.7491
Epoch 2: val_loss improved from 0.77192 to 0.72640, saving model to model_adding_ver1/vgg/best_model.h5
83/83 [==============================] - 23s 278ms/step - loss: 0.6664 - accuracy: 0.7491 - val_loss: 0.7264 - val_accuracy: 0.7687
Epoch 3/50
83/83 [==============================] - ETA: 0s - loss: 0.5502 - accuracy: 0.8086
Epoch 3: val_loss improved from 0.72640 to 0.61201, saving model to model_adding_ver1/vgg/best_model.h5
83/83 [==============================] - 23s 279ms/step - loss: 0.5502 - accuracy: 0.8086 - val_loss: 0.6120 - val_accuracy: 0.7789
Epoch 4/50
83/83 [=======

83/83 [==============================] - 19s 227ms/step - loss: 0.4710 - accuracy: 0.8173 - val_loss: 0.4594 - val_accuracy: 0.8061
Epoch 5/50
83/83 [==============================] - ETA: 0s - loss: 0.4084 - accuracy: 0.8483
Epoch 5: val_loss improved from 0.45942 to 0.37424, saving model to model_adding_ver1/resnet/best_model.h5
83/83 [==============================] - 27s 319ms/step - loss: 0.4084 - accuracy: 0.8483 - val_loss: 0.3742 - val_accuracy: 0.8673
Epoch 6/50
83/83 [==============================] - ETA: 0s - loss: 0.3424 - accuracy: 0.8842
Epoch 6: val_loss did not improve from 0.37424
83/83 [==============================] - 17s 205ms/step - loss: 0.3424 - accuracy: 0.8842 - val_loss: 0.3776 - val_accuracy: 0.8639
Epoch 7/50
83/83 [==============================] - ETA: 0s - loss: 0.3049 - accuracy: 0.8990
Epoch 7: val_loss improved from 0.37424 to 0.32868, saving model to model_adding_ver1/resnet/best_model.h5
83/83 [==============================] - 16s 189ms/step - los

Epoch 31/50
83/83 [==============================] - ETA: 0s - loss: 0.0400 - accuracy: 0.9981
Epoch 31: val_loss did not improve from 0.17352
83/83 [==============================] - 14s 174ms/step - loss: 0.0400 - accuracy: 0.9981 - val_loss: 0.1804 - val_accuracy: 0.9354
Epoch 32/50
83/83 [==============================] - ETA: 0s - loss: 0.0400 - accuracy: 0.9977
Epoch 32: val_loss did not improve from 0.17352
83/83 [==============================] - 15s 175ms/step - loss: 0.0400 - accuracy: 0.9977 - val_loss: 0.1829 - val_accuracy: 0.9388
Epoch 33/50
83/83 [==============================] - ETA: 0s - loss: 0.0364 - accuracy: 0.9981
Epoch 33: val_loss did not improve from 0.17352
83/83 [==============================] - 15s 176ms/step - loss: 0.0364 - accuracy: 0.9981 - val_loss: 0.1790 - val_accuracy: 0.9422
Epoch 34/50
83/83 [==============================] - ETA: 0s - loss: 0.0363 - accuracy: 0.9974
Epoch 34: val_loss did not improve from 0.17352
83/83 [=========================

In [10]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import os

epochs=50

version = "model_adding_ver2"
def create_model(base_model, output_size, model_name):
    model_dir = f"{version}/{model_name}/"
    for layer in base_model.layers:
        layer.trainable = False
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(rate=0.3)(x)
    output = Dense(output_size, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)
    best_model_checkpoint = ModelCheckpoint(f"{model_dir}best_model.h5", monitor='val_loss', save_best_only=True, mode='min', verbose=1)
    csv_logger = CSVLogger(f'training_log_{model_name}.csv', separator=',', append=False)
    return model, [early_stopping, best_model_checkpoint, csv_logger]


base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_vgg, callbacks_vgg = create_model(base_model_vgg, output_size=4, model_name='vgg')

base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_resnet, callbacks_resnet = create_model(base_model_resnet, output_size=4, model_name='resnet')

history_vgg = model_vgg.fit(X_train, y_train,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            verbose=1,
                            callbacks=callbacks_vgg)

history_resnet = model_resnet.fit(X_train, y_train,
                                  epochs=epochs,
                                  validation_data=(X_val, y_val),
                                  verbose=1,
                                  callbacks=callbacks_resnet)


from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

for model_name in ["resnet", "vgg"]:
    best_model = keras.models.load_model(version+"/"+model_name+"/best_model.h5")
    y_pred = best_model.predict(X_test)
    y_pred_single_label = np.argmax(y_pred, axis=1)
    y_test_single_label = np.argmax(y_test, axis=1)
    precision = precision_score(y_test_single_label, y_pred_single_label, average='macro')
    recall = recall_score(y_test_single_label, y_pred_single_label, average='macro')
    f1 = f1_score(y_test_single_label, y_pred_single_label, average='macro')
    accuracy = accuracy_score(y_test_single_label, y_pred_single_label)
    precision = round(precision, 3)
    recall = round(recall, 3)
    f1 = round(f1, 3)
    accuracy = round(accuracy, 3)
    print(str(model_name))
    print(precision, recall, f1, accuracy)

Epoch 1/50
83/83 [==============================] - ETA: 0s - loss: 1.0339 - accuracy: 0.5668
Epoch 1: val_loss improved from inf to 0.87202, saving model to model_adding_ver2/vgg/best_model.h5
83/83 [==============================] - 30s 298ms/step - loss: 1.0339 - accuracy: 0.5668 - val_loss: 0.8720 - val_accuracy: 0.6156
Epoch 2/50
83/83 [==============================] - ETA: 0s - loss: 0.7097 - accuracy: 0.7344
Epoch 2: val_loss improved from 0.87202 to 0.69884, saving model to model_adding_ver2/vgg/best_model.h5
83/83 [==============================] - 23s 273ms/step - loss: 0.7097 - accuracy: 0.7344 - val_loss: 0.6988 - val_accuracy: 0.7041
Epoch 3/50
83/83 [==============================] - ETA: 0s - loss: 0.5902 - accuracy: 0.7836
Epoch 3: val_loss improved from 0.69884 to 0.57193, saving model to model_adding_ver2/vgg/best_model.h5
83/83 [==============================] - 23s 276ms/step - loss: 0.5902 - accuracy: 0.7836 - val_loss: 0.5719 - val_accuracy: 0.7993
Epoch 4/50
83/

Epoch 28/50
83/83 [==============================] - ETA: 0s - loss: 0.1487 - accuracy: 0.9637
Epoch 28: val_loss improved from 0.29173 to 0.27189, saving model to model_adding_ver2/vgg/best_model.h5
83/83 [==============================] - 23s 277ms/step - loss: 0.1487 - accuracy: 0.9637 - val_loss: 0.2719 - val_accuracy: 0.9048
Epoch 29/50
83/83 [==============================] - ETA: 0s - loss: 0.1354 - accuracy: 0.9720
Epoch 29: val_loss improved from 0.27189 to 0.26128, saving model to model_adding_ver2/vgg/best_model.h5
83/83 [==============================] - 23s 274ms/step - loss: 0.1354 - accuracy: 0.9720 - val_loss: 0.2613 - val_accuracy: 0.9116
Epoch 30/50
83/83 [==============================] - ETA: 0s - loss: 0.1344 - accuracy: 0.9735
Epoch 30: val_loss improved from 0.26128 to 0.25471, saving model to model_adding_ver2/vgg/best_model.h5
83/83 [==============================] - 23s 273ms/step - loss: 0.1344 - accuracy: 0.9735 - val_loss: 0.2547 - val_accuracy: 0.9150
Epoc

83/83 [==============================] - 16s 193ms/step - loss: 0.0592 - accuracy: 0.9924 - val_loss: 0.1864 - val_accuracy: 0.9218
Epoch 33/50
83/83 [==============================] - ETA: 0s - loss: 0.0546 - accuracy: 0.9928
Epoch 33: val_loss did not improve from 0.18643
83/83 [==============================] - 15s 179ms/step - loss: 0.0546 - accuracy: 0.9928 - val_loss: 0.1955 - val_accuracy: 0.9150
Epoch 34/50
83/83 [==============================] - ETA: 0s - loss: 0.0522 - accuracy: 0.9939
Epoch 34: val_loss did not improve from 0.18643
83/83 [==============================] - 16s 187ms/step - loss: 0.0522 - accuracy: 0.9939 - val_loss: 0.1930 - val_accuracy: 0.9422
Epoch 35/50
83/83 [==============================] - ETA: 0s - loss: 0.0520 - accuracy: 0.9924
Epoch 35: val_loss did not improve from 0.18643
83/83 [==============================] - 15s 182ms/step - loss: 0.0520 - accuracy: 0.9924 - val_loss: 0.1896 - val_accuracy: 0.9286
Epoch 36/50
83/83 [========================

In [11]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import os

epochs=50

version = "model_adding_ver3"
def create_model(base_model, output_size, model_name):
    model_dir = f"{version}/{model_name}/"
    for layer in base_model.layers:
        layer.trainable = False
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(output_size, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)
    best_model_checkpoint = ModelCheckpoint(f"{model_dir}best_model.h5", monitor='val_loss', save_best_only=True, mode='min', verbose=1)
    csv_logger = CSVLogger(f'training_log_{model_name}.csv', separator=',', append=False)
    return model, [early_stopping, best_model_checkpoint, csv_logger]


base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_vgg, callbacks_vgg = create_model(base_model_vgg, output_size=4, model_name='vgg')

base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_resnet, callbacks_resnet = create_model(base_model_resnet, output_size=4, model_name='resnet')


history_vgg = model_vgg.fit(X_train, y_train,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            verbose=1,
                            callbacks=callbacks_vgg)

history_resnet = model_resnet.fit(X_train, y_train,
                                  epochs=epochs,
                                  validation_data=(X_val, y_val),
                                  verbose=1,
                                  callbacks=callbacks_resnet)


from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

for model_name in ["resnet", "vgg"]:
    best_model = keras.models.load_model(version+"/"+model_name+"/best_model.h5")
    y_pred = best_model.predict(X_test)
    y_pred_single_label = np.argmax(y_pred, axis=1)
    y_test_single_label = np.argmax(y_test, axis=1)
    precision = precision_score(y_test_single_label, y_pred_single_label, average='macro')
    recall = recall_score(y_test_single_label, y_pred_single_label, average='macro')
    f1 = f1_score(y_test_single_label, y_pred_single_label, average='macro')
    accuracy = accuracy_score(y_test_single_label, y_pred_single_label)
    precision = round(precision, 3)
    recall = round(recall, 3)
    f1 = round(f1, 3)
    accuracy = round(accuracy, 3)
    print(str(model_name))
    print(precision, recall, f1, accuracy)

Epoch 1/50
83/83 [==============================] - ETA: 0s - loss: 1.0130 - accuracy: 0.5963
Epoch 1: val_loss improved from inf to 0.86842, saving model to model_adding_ver3/vgg/best_model.h5
83/83 [==============================] - 25s 293ms/step - loss: 1.0130 - accuracy: 0.5963 - val_loss: 0.8684 - val_accuracy: 0.6395
Epoch 2/50
83/83 [==============================] - ETA: 0s - loss: 0.6993 - accuracy: 0.7450
Epoch 2: val_loss improved from 0.86842 to 0.67510, saving model to model_adding_ver3/vgg/best_model.h5
83/83 [==============================] - 22s 267ms/step - loss: 0.6993 - accuracy: 0.7450 - val_loss: 0.6751 - val_accuracy: 0.7653
Epoch 3/50
83/83 [==============================] - ETA: 0s - loss: 0.5782 - accuracy: 0.7915
Epoch 3: val_loss improved from 0.67510 to 0.58847, saving model to model_adding_ver3/vgg/best_model.h5
83/83 [==============================] - 22s 268ms/step - loss: 0.5782 - accuracy: 0.7915 - val_loss: 0.5885 - val_accuracy: 0.7857
Epoch 4/50
83/

Epoch 28/50
83/83 [==============================] - ETA: 0s - loss: 0.1049 - accuracy: 0.9830
Epoch 28: val_loss improved from 0.24122 to 0.23256, saving model to model_adding_ver3/vgg/best_model.h5
83/83 [==============================] - 22s 267ms/step - loss: 0.1049 - accuracy: 0.9830 - val_loss: 0.2326 - val_accuracy: 0.9354
Epoch 29/50
83/83 [==============================] - ETA: 0s - loss: 0.1030 - accuracy: 0.9818
Epoch 29: val_loss improved from 0.23256 to 0.22438, saving model to model_adding_ver3/vgg/best_model.h5
83/83 [==============================] - 28s 336ms/step - loss: 0.1030 - accuracy: 0.9818 - val_loss: 0.2244 - val_accuracy: 0.9252
Epoch 30/50
83/83 [==============================] - ETA: 0s - loss: 0.0982 - accuracy: 0.9860
Epoch 30: val_loss improved from 0.22438 to 0.22017, saving model to model_adding_ver3/vgg/best_model.h5
83/83 [==============================] - 26s 313ms/step - loss: 0.0982 - accuracy: 0.9860 - val_loss: 0.2202 - val_accuracy: 0.9252
Epoc

Epoch 5/50
83/83 [==============================] - ETA: 0s - loss: 0.3949 - accuracy: 0.8528
Epoch 5: val_loss improved from 0.43985 to 0.40420, saving model to model_adding_ver3/resnet/best_model.h5
83/83 [==============================] - 16s 192ms/step - loss: 0.3949 - accuracy: 0.8528 - val_loss: 0.4042 - val_accuracy: 0.8537
Epoch 6/50
83/83 [==============================] - ETA: 0s - loss: 0.3196 - accuracy: 0.8808
Epoch 6: val_loss did not improve from 0.40420
83/83 [==============================] - 15s 178ms/step - loss: 0.3196 - accuracy: 0.8808 - val_loss: 0.5193 - val_accuracy: 0.7925
Epoch 7/50
83/83 [==============================] - ETA: 0s - loss: 0.3117 - accuracy: 0.8903
Epoch 7: val_loss did not improve from 0.40420
83/83 [==============================] - 15s 184ms/step - loss: 0.3117 - accuracy: 0.8903 - val_loss: 0.4331 - val_accuracy: 0.8061
Epoch 8/50
83/83 [==============================] - ETA: 0s - loss: 0.2526 - accuracy: 0.9183
Epoch 8: val_loss improved 

Epoch 32/50
83/83 [==============================] - ETA: 0s - loss: 0.0307 - accuracy: 0.9989
Epoch 32: val_loss did not improve from 0.20027
83/83 [==============================] - 15s 179ms/step - loss: 0.0307 - accuracy: 0.9989 - val_loss: 0.2256 - val_accuracy: 0.9150
Epoch 33/50
83/83 [==============================] - ETA: 0s - loss: 0.0293 - accuracy: 0.9985
Epoch 33: val_loss improved from 0.20027 to 0.19379, saving model to model_adding_ver3/resnet/best_model.h5
83/83 [==============================] - 16s 190ms/step - loss: 0.0293 - accuracy: 0.9985 - val_loss: 0.1938 - val_accuracy: 0.9218
Epoch 34/50
83/83 [==============================] - ETA: 0s - loss: 0.0282 - accuracy: 0.9992
Epoch 34: val_loss did not improve from 0.19379
83/83 [==============================] - 15s 175ms/step - loss: 0.0282 - accuracy: 0.9992 - val_loss: 0.1941 - val_accuracy: 0.9252
Epoch 35/50
83/83 [==============================] - ETA: 0s - loss: 0.0255 - accuracy: 0.9989
Epoch 35: val_loss d

In [12]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import os

epochs=50

version = "model_adding_ver4"
def create_model(base_model, output_size, model_name):
    model_dir = f"{version}/{model_name}/"
    for layer in base_model.layers:
        layer.trainable = False
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(rate=0.3)(x)

    output = Dense(output_size, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)
    best_model_checkpoint = ModelCheckpoint(f"{model_dir}best_model.h5", monitor='val_loss', save_best_only=True, mode='min', verbose=1)
    csv_logger = CSVLogger(f'training_log_{model_name}.csv', separator=',', append=False)
    return model, [early_stopping, best_model_checkpoint, csv_logger]


base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_vgg, callbacks_vgg = create_model(base_model_vgg, output_size=4, model_name='vgg')

base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_resnet, callbacks_resnet = create_model(base_model_resnet, output_size=4, model_name='resnet')


history_vgg = model_vgg.fit(X_train, y_train,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            verbose=1,
                            callbacks=callbacks_vgg)

history_resnet = model_resnet.fit(X_train, y_train,
                                  epochs=epochs,
                                  validation_data=(X_val, y_val),
                                  verbose=1,
                                  callbacks=callbacks_resnet)


from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

for model_name in ["resnet", "vgg"]:
    best_model = keras.models.load_model(version+"/"+model_name+"/best_model.h5")
    y_pred = best_model.predict(X_test)
    y_pred_single_label = np.argmax(y_pred, axis=1)
    y_test_single_label = np.argmax(y_test, axis=1)
    precision = precision_score(y_test_single_label, y_pred_single_label, average='macro')
    recall = recall_score(y_test_single_label, y_pred_single_label, average='macro')
    f1 = f1_score(y_test_single_label, y_pred_single_label, average='macro')
    accuracy = accuracy_score(y_test_single_label, y_pred_single_label)
    precision = round(precision, 3)
    recall = round(recall, 3)
    f1 = round(f1, 3)
    accuracy = round(accuracy, 3)
    print(str(model_name))
    print(precision, recall, f1, accuracy)

Epoch 1/50
83/83 [==============================] - ETA: 0s - loss: 1.0733 - accuracy: 0.5407
Epoch 1: val_loss improved from inf to 0.84771, saving model to model_adding_ver4/vgg/best_model.h5
83/83 [==============================] - 25s 291ms/step - loss: 1.0733 - accuracy: 0.5407 - val_loss: 0.8477 - val_accuracy: 0.7041
Epoch 2/50
83/83 [==============================] - ETA: 0s - loss: 0.7889 - accuracy: 0.7026
Epoch 2: val_loss improved from 0.84771 to 0.75232, saving model to model_adding_ver4/vgg/best_model.h5
83/83 [==============================] - 22s 265ms/step - loss: 0.7889 - accuracy: 0.7026 - val_loss: 0.7523 - val_accuracy: 0.7381
Epoch 3/50
83/83 [==============================] - ETA: 0s - loss: 0.6385 - accuracy: 0.7722
Epoch 3: val_loss improved from 0.75232 to 0.68616, saving model to model_adding_ver4/vgg/best_model.h5
83/83 [==============================] - 22s 270ms/step - loss: 0.6385 - accuracy: 0.7722 - val_loss: 0.6862 - val_accuracy: 0.7551
Epoch 4/50
83/

Epoch 4/50
83/83 [==============================] - ETA: 0s - loss: 0.5389 - accuracy: 0.7896
Epoch 4: val_loss did not improve from 0.51020
83/83 [==============================] - 15s 176ms/step - loss: 0.5389 - accuracy: 0.7896 - val_loss: 0.5654 - val_accuracy: 0.7517
Epoch 5/50
83/83 [==============================] - ETA: 0s - loss: 0.4724 - accuracy: 0.8180
Epoch 5: val_loss improved from 0.51020 to 0.45889, saving model to model_adding_ver4/resnet/best_model.h5
83/83 [==============================] - 16s 189ms/step - loss: 0.4724 - accuracy: 0.8180 - val_loss: 0.4589 - val_accuracy: 0.8163
Epoch 6/50
83/83 [==============================] - ETA: 0s - loss: 0.3991 - accuracy: 0.8555
Epoch 6: val_loss improved from 0.45889 to 0.45405, saving model to model_adding_ver4/resnet/best_model.h5
83/83 [==============================] - 15s 184ms/step - loss: 0.3991 - accuracy: 0.8555 - val_loss: 0.4541 - val_accuracy: 0.8197
Epoch 7/50
83/83 [==============================] - ETA: 0s -

83/83 [==============================] - 15s 182ms/step - loss: 0.0429 - accuracy: 0.9943 - val_loss: 0.1904 - val_accuracy: 0.9184
Epoch 32/50
83/83 [==============================] - ETA: 0s - loss: 0.0490 - accuracy: 0.9932
Epoch 32: val_loss did not improve from 0.19040
83/83 [==============================] - 14s 173ms/step - loss: 0.0490 - accuracy: 0.9932 - val_loss: 0.1950 - val_accuracy: 0.9320
Epoch 33/50
83/83 [==============================] - ETA: 0s - loss: 0.0380 - accuracy: 0.9962
Epoch 33: val_loss did not improve from 0.19040
83/83 [==============================] - 15s 177ms/step - loss: 0.0380 - accuracy: 0.9962 - val_loss: 0.2059 - val_accuracy: 0.9184
Epoch 34/50
83/83 [==============================] - ETA: 0s - loss: 0.0412 - accuracy: 0.9936
Epoch 34: val_loss did not improve from 0.19040
83/83 [==============================] - 15s 177ms/step - loss: 0.0412 - accuracy: 0.9936 - val_loss: 0.2103 - val_accuracy: 0.9116
Epoch 35/50
83/83 [========================

In [13]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import os

epochs=50

version = "model_adding_ver5"
def create_model(base_model, output_size, model_name):
    model_dir = f"{version}/{model_name}/"
    for layer in base_model.layers:
        layer.trainable = False
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    output = Dense(output_size, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)
    best_model_checkpoint = ModelCheckpoint(f"{model_dir}best_model.h5", monitor='val_loss', save_best_only=True, mode='min', verbose=1)
    csv_logger = CSVLogger(f'training_log_{model_name}.csv', separator=',', append=False)
    return model, [early_stopping, best_model_checkpoint, csv_logger]


base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_vgg, callbacks_vgg = create_model(base_model_vgg, output_size=4, model_name='vgg')

base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_resnet, callbacks_resnet = create_model(base_model_resnet, output_size=4, model_name='resnet')


history_vgg = model_vgg.fit(X_train, y_train,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            verbose=1,
                            callbacks=callbacks_vgg)

history_resnet = model_resnet.fit(X_train, y_train,
                                  epochs=epochs,
                                  validation_data=(X_val, y_val),
                                  verbose=1,
                                  callbacks=callbacks_resnet)


from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

for model_name in ["resnet", "vgg"]:
    best_model = keras.models.load_model(version+"/"+model_name+"/best_model.h5")
    y_pred = best_model.predict(X_test)
    y_pred_single_label = np.argmax(y_pred, axis=1)
    y_test_single_label = np.argmax(y_test, axis=1)
    precision = precision_score(y_test_single_label, y_pred_single_label, average='macro')
    recall = recall_score(y_test_single_label, y_pred_single_label, average='macro')
    f1 = f1_score(y_test_single_label, y_pred_single_label, average='macro')
    accuracy = accuracy_score(y_test_single_label, y_pred_single_label)
    precision = round(precision, 3)
    recall = round(recall, 3)
    f1 = round(f1, 3)
    accuracy = round(accuracy, 3)
    print(str(model_name))
    print(precision, recall, f1, accuracy)

Epoch 1/50
83/83 [==============================] - ETA: 0s - loss: 1.0836 - accuracy: 0.5460
Epoch 1: val_loss improved from inf to 1.00648, saving model to model_adding_ver5/vgg/best_model.h5
83/83 [==============================] - 26s 297ms/step - loss: 1.0836 - accuracy: 0.5460 - val_loss: 1.0065 - val_accuracy: 0.5374
Epoch 2/50
83/83 [==============================] - ETA: 0s - loss: 0.7511 - accuracy: 0.7367
Epoch 2: val_loss improved from 1.00648 to 0.75436, saving model to model_adding_ver5/vgg/best_model.h5
83/83 [==============================] - 22s 266ms/step - loss: 0.7511 - accuracy: 0.7367 - val_loss: 0.7544 - val_accuracy: 0.7109
Epoch 3/50
83/83 [==============================] - ETA: 0s - loss: 0.6196 - accuracy: 0.7923
Epoch 3: val_loss improved from 0.75436 to 0.66152, saving model to model_adding_ver5/vgg/best_model.h5
83/83 [==============================] - 22s 268ms/step - loss: 0.6196 - accuracy: 0.7923 - val_loss: 0.6615 - val_accuracy: 0.7687
Epoch 4/50
83/

Epoch 28/50
83/83 [==============================] - ETA: 0s - loss: 0.1114 - accuracy: 0.9758
Epoch 28: val_loss improved from 0.24517 to 0.23924, saving model to model_adding_ver5/vgg/best_model.h5
83/83 [==============================] - 22s 269ms/step - loss: 0.1114 - accuracy: 0.9758 - val_loss: 0.2392 - val_accuracy: 0.9048
Epoch 29/50
83/83 [==============================] - ETA: 0s - loss: 0.1023 - accuracy: 0.9811
Epoch 29: val_loss did not improve from 0.23924
83/83 [==============================] - 22s 266ms/step - loss: 0.1023 - accuracy: 0.9811 - val_loss: 0.2489 - val_accuracy: 0.9116
Epoch 30/50
83/83 [==============================] - ETA: 0s - loss: 0.1084 - accuracy: 0.9758
Epoch 30: val_loss did not improve from 0.23924
83/83 [==============================] - 22s 266ms/step - loss: 0.1084 - accuracy: 0.9758 - val_loss: 0.2397 - val_accuracy: 0.9048
Epoch 31/50
83/83 [==============================] - ETA: 0s - loss: 0.0906 - accuracy: 0.9879
Epoch 31: val_loss impr

83/83 [==============================] - ETA: 0s - loss: 0.0213 - accuracy: 0.9989
Epoch 32: val_loss did not improve from 0.18391
83/83 [==============================] - 15s 175ms/step - loss: 0.0213 - accuracy: 0.9989 - val_loss: 0.3870 - val_accuracy: 0.8469
Epoch 33/50
83/83 [==============================] - ETA: 0s - loss: 0.0228 - accuracy: 0.9974
Epoch 33: val_loss did not improve from 0.18391
83/83 [==============================] - 15s 179ms/step - loss: 0.0228 - accuracy: 0.9974 - val_loss: 0.2233 - val_accuracy: 0.9048
Epoch 34/50
83/83 [==============================] - ETA: 0s - loss: 0.0199 - accuracy: 0.9992
Epoch 34: val_loss did not improve from 0.18391
83/83 [==============================] - 15s 178ms/step - loss: 0.0199 - accuracy: 0.9992 - val_loss: 0.1898 - val_accuracy: 0.9320
Epoch 35/50
83/83 [==============================] - ETA: 0s - loss: 0.0200 - accuracy: 0.9989
Epoch 35: val_loss improved from 0.18391 to 0.17673, saving model to model_adding_ver5/resne

In [14]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import os

epochs=50

version = "model_adding_ver6"
def create_model(base_model, output_size, model_name):
    model_dir = f"{version}/{model_name}/"
    for layer in base_model.layers:
        layer.trainable = False
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(rate=0.3)(x)

    output = Dense(output_size, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)
    best_model_checkpoint = ModelCheckpoint(f"{model_dir}best_model.h5", monitor='val_loss', save_best_only=True, mode='min', verbose=1)
    csv_logger = CSVLogger(f'training_log_{model_name}.csv', separator=',', append=False)
    return model, [early_stopping, best_model_checkpoint, csv_logger]


base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_vgg, callbacks_vgg = create_model(base_model_vgg, output_size=4, model_name='vgg')

base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_resnet, callbacks_resnet = create_model(base_model_resnet, output_size=4, model_name='resnet')


history_vgg = model_vgg.fit(X_train, y_train,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            verbose=1,
                            callbacks=callbacks_vgg)

history_resnet = model_resnet.fit(X_train, y_train,
                                  epochs=epochs,
                                  validation_data=(X_val, y_val),
                                  verbose=1,
                                  callbacks=callbacks_resnet)


from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

for model_name in ["resnet", "vgg"]:
    best_model = keras.models.load_model(version+"/"+model_name+"/best_model.h5")
    y_pred = best_model.predict(X_test)
    y_pred_single_label = np.argmax(y_pred, axis=1)
    y_test_single_label = np.argmax(y_test, axis=1)
    precision = precision_score(y_test_single_label, y_pred_single_label, average='macro')
    recall = recall_score(y_test_single_label, y_pred_single_label, average='macro')
    f1 = f1_score(y_test_single_label, y_pred_single_label, average='macro')
    accuracy = accuracy_score(y_test_single_label, y_pred_single_label)
    precision = round(precision, 3)
    recall = round(recall, 3)
    f1 = round(f1, 3)
    accuracy = round(accuracy, 3)
    print(str(model_name))
    print(precision, recall, f1, accuracy)

Epoch 1/50
83/83 [==============================] - ETA: 0s - loss: 1.1224 - accuracy: 0.5259
Epoch 1: val_loss improved from inf to 0.94198, saving model to model_adding_ver6/vgg/best_model.h5
83/83 [==============================] - 26s 290ms/step - loss: 1.1224 - accuracy: 0.5259 - val_loss: 0.9420 - val_accuracy: 0.6633
Epoch 2/50
83/83 [==============================] - ETA: 0s - loss: 0.8599 - accuracy: 0.6625
Epoch 2: val_loss improved from 0.94198 to 0.79960, saving model to model_adding_ver6/vgg/best_model.h5
83/83 [==============================] - 22s 266ms/step - loss: 0.8599 - accuracy: 0.6625 - val_loss: 0.7996 - val_accuracy: 0.7347
Epoch 3/50
83/83 [==============================] - ETA: 0s - loss: 0.7412 - accuracy: 0.7196
Epoch 3: val_loss improved from 0.79960 to 0.69509, saving model to model_adding_ver6/vgg/best_model.h5
83/83 [==============================] - 23s 273ms/step - loss: 0.7412 - accuracy: 0.7196 - val_loss: 0.6951 - val_accuracy: 0.7347
Epoch 4/50
83/

83/83 [==============================] - 22s 267ms/step - loss: 0.1430 - accuracy: 0.9629 - val_loss: 0.3069 - val_accuracy: 0.8810
Epoch 28/50
83/83 [==============================] - ETA: 0s - loss: 0.1473 - accuracy: 0.9595
Epoch 28: val_loss improved from 0.29050 to 0.24842, saving model to model_adding_ver6/vgg/best_model.h5
83/83 [==============================] - 22s 268ms/step - loss: 0.1473 - accuracy: 0.9595 - val_loss: 0.2484 - val_accuracy: 0.9048
Epoch 29/50
83/83 [==============================] - ETA: 0s - loss: 0.1411 - accuracy: 0.9610
Epoch 29: val_loss did not improve from 0.24842
83/83 [==============================] - 22s 265ms/step - loss: 0.1411 - accuracy: 0.9610 - val_loss: 0.2731 - val_accuracy: 0.8980
Epoch 30/50
83/83 [==============================] - ETA: 0s - loss: 0.1278 - accuracy: 0.9712
Epoch 30: val_loss did not improve from 0.24842
83/83 [==============================] - 22s 267ms/step - loss: 0.1278 - accuracy: 0.9712 - val_loss: 0.2593 - val_acc

Epoch 5/50
83/83 [==============================] - ETA: 0s - loss: 0.5248 - accuracy: 0.7915
Epoch 5: val_loss improved from 0.56174 to 0.53168, saving model to model_adding_ver6/resnet/best_model.h5
83/83 [==============================] - 16s 188ms/step - loss: 0.5248 - accuracy: 0.7915 - val_loss: 0.5317 - val_accuracy: 0.7891
Epoch 6/50
83/83 [==============================] - ETA: 0s - loss: 0.4433 - accuracy: 0.8434
Epoch 6: val_loss improved from 0.53168 to 0.41045, saving model to model_adding_ver6/resnet/best_model.h5
83/83 [==============================] - 16s 190ms/step - loss: 0.4433 - accuracy: 0.8434 - val_loss: 0.4104 - val_accuracy: 0.8571
Epoch 7/50
83/83 [==============================] - ETA: 0s - loss: 0.4030 - accuracy: 0.8513
Epoch 7: val_loss improved from 0.41045 to 0.40612, saving model to model_adding_ver6/resnet/best_model.h5
83/83 [==============================] - 16s 187ms/step - loss: 0.4030 - accuracy: 0.8513 - val_loss: 0.4061 - val_accuracy: 0.8571
E

83/83 [==============================] - 16s 189ms/step - loss: 0.0509 - accuracy: 0.9902 - val_loss: 0.2068 - val_accuracy: 0.9354
Epoch 33/50
83/83 [==============================] - ETA: 0s - loss: 0.0461 - accuracy: 0.9924
Epoch 33: val_loss did not improve from 0.20683
83/83 [==============================] - 15s 175ms/step - loss: 0.0461 - accuracy: 0.9924 - val_loss: 0.2787 - val_accuracy: 0.9048
Epoch 34/50
83/83 [==============================] - ETA: 0s - loss: 0.0403 - accuracy: 0.9936
Epoch 34: val_loss improved from 0.20683 to 0.19539, saving model to model_adding_ver6/resnet/best_model.h5
83/83 [==============================] - 15s 187ms/step - loss: 0.0403 - accuracy: 0.9936 - val_loss: 0.1954 - val_accuracy: 0.9252
Epoch 35/50
83/83 [==============================] - ETA: 0s - loss: 0.0391 - accuracy: 0.9924
Epoch 35: val_loss improved from 0.19539 to 0.19143, saving model to model_adding_ver6/resnet/best_model.h5
83/83 [==============================] - 15s 185ms/step

In [15]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import os

epochs=50

version = "model_adding_ver7"
def create_model(base_model, output_size, model_name):
    model_dir = f"{version}/{model_name}/"
    for layer in base_model.layers:
        layer.trainable = False
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)

    output = Dense(output_size, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)
    best_model_checkpoint = ModelCheckpoint(f"{model_dir}best_model.h5", monitor='val_loss', save_best_only=True, mode='min', verbose=1)
    csv_logger = CSVLogger(f'training_log_{model_name}.csv', separator=',', append=False)
    return model, [early_stopping, best_model_checkpoint, csv_logger]


base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_vgg, callbacks_vgg = create_model(base_model_vgg, output_size=4, model_name='vgg')

base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_resnet, callbacks_resnet = create_model(base_model_resnet, output_size=4, model_name='resnet')


history_vgg = model_vgg.fit(X_train, y_train,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            verbose=1,
                            callbacks=callbacks_vgg)

history_resnet = model_resnet.fit(X_train, y_train,
                                  epochs=epochs,
                                  validation_data=(X_val, y_val),
                                  verbose=1,
                                  callbacks=callbacks_resnet)


from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

for model_name in ["resnet", "vgg"]:
    best_model = keras.models.load_model(version+"/"+model_name+"/best_model.h5")
    y_pred = best_model.predict(X_test)
    y_pred_single_label = np.argmax(y_pred, axis=1)
    y_test_single_label = np.argmax(y_test, axis=1)
    precision = precision_score(y_test_single_label, y_pred_single_label, average='macro')
    recall = recall_score(y_test_single_label, y_pred_single_label, average='macro')
    f1 = f1_score(y_test_single_label, y_pred_single_label, average='macro')
    accuracy = accuracy_score(y_test_single_label, y_pred_single_label)
    precision = round(precision, 3)
    recall = round(recall, 3)
    f1 = round(f1, 3)
    accuracy = round(accuracy, 3)
    print(str(model_name))
    print(precision, recall, f1, accuracy)

Epoch 1/50
83/83 [==============================] - ETA: 0s - loss: 1.1384 - accuracy: 0.5218
Epoch 1: val_loss improved from inf to 1.01087, saving model to model_adding_ver7/vgg/best_model.h5
83/83 [==============================] - 27s 292ms/step - loss: 1.1384 - accuracy: 0.5218 - val_loss: 1.0109 - val_accuracy: 0.6122
Epoch 2/50
83/83 [==============================] - ETA: 0s - loss: 0.8223 - accuracy: 0.7003
Epoch 2: val_loss improved from 1.01087 to 0.77658, saving model to model_adding_ver7/vgg/best_model.h5
83/83 [==============================] - 22s 263ms/step - loss: 0.8223 - accuracy: 0.7003 - val_loss: 0.7766 - val_accuracy: 0.7313
Epoch 3/50
83/83 [==============================] - ETA: 0s - loss: 0.6837 - accuracy: 0.7575
Epoch 3: val_loss improved from 0.77658 to 0.68892, saving model to model_adding_ver7/vgg/best_model.h5
83/83 [==============================] - 22s 269ms/step - loss: 0.6837 - accuracy: 0.7575 - val_loss: 0.6889 - val_accuracy: 0.7823
Epoch 4/50
83/

Epoch 27/50
83/83 [==============================] - ETA: 0s - loss: 0.1139 - accuracy: 0.9731
Epoch 27: val_loss did not improve from 0.23821
83/83 [==============================] - 22s 265ms/step - loss: 0.1139 - accuracy: 0.9731 - val_loss: 0.2574 - val_accuracy: 0.9082
Epoch 28/50
83/83 [==============================] - ETA: 0s - loss: 0.1111 - accuracy: 0.9758
Epoch 28: val_loss did not improve from 0.23821
83/83 [==============================] - 22s 266ms/step - loss: 0.1111 - accuracy: 0.9758 - val_loss: 0.2528 - val_accuracy: 0.9252
Epoch 29/50
83/83 [==============================] - ETA: 0s - loss: 0.1009 - accuracy: 0.9803
Epoch 29: val_loss did not improve from 0.23821
83/83 [==============================] - 22s 267ms/step - loss: 0.1009 - accuracy: 0.9803 - val_loss: 0.2477 - val_accuracy: 0.9252
Epoch 30/50
83/83 [==============================] - ETA: 0s - loss: 0.0961 - accuracy: 0.9781
Epoch 30: val_loss did not improve from 0.23821
83/83 [=========================

83/83 [==============================] - 16s 187ms/step - loss: 0.5049 - accuracy: 0.8082 - val_loss: 0.6480 - val_accuracy: 0.7449
Epoch 5/50
83/83 [==============================] - ETA: 0s - loss: 0.4196 - accuracy: 0.8490
Epoch 5: val_loss improved from 0.64805 to 0.55873, saving model to model_adding_ver7/resnet/best_model.h5
83/83 [==============================] - 16s 190ms/step - loss: 0.4196 - accuracy: 0.8490 - val_loss: 0.5587 - val_accuracy: 0.7279
Epoch 6/50
83/83 [==============================] - ETA: 0s - loss: 0.3778 - accuracy: 0.8574
Epoch 6: val_loss improved from 0.55873 to 0.41189, saving model to model_adding_ver7/resnet/best_model.h5
83/83 [==============================] - 16s 189ms/step - loss: 0.3778 - accuracy: 0.8574 - val_loss: 0.4119 - val_accuracy: 0.8401
Epoch 7/50
83/83 [==============================] - ETA: 0s - loss: 0.3127 - accuracy: 0.8952
Epoch 7: val_loss improved from 0.41189 to 0.34855, saving model to model_adding_ver7/resnet/best_model.h5
8

In [16]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import os

epochs=50

version = "model_adding_ver8"
def create_model(base_model, output_size, model_name):
    model_dir = f"{version}/{model_name}/"
    for layer in base_model.layers:
        layer.trainable = False
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(rate=0.3)(x)

    output = Dense(output_size, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)
    best_model_checkpoint = ModelCheckpoint(f"{model_dir}best_model.h5", monitor='val_loss', save_best_only=True, mode='min', verbose=1)
    csv_logger = CSVLogger(f'training_log_{model_name}.csv', separator=',', append=False)
    return model, [early_stopping, best_model_checkpoint, csv_logger]


base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_vgg, callbacks_vgg = create_model(base_model_vgg, output_size=4, model_name='vgg')

base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_resnet, callbacks_resnet = create_model(base_model_resnet, output_size=4, model_name='resnet')


history_vgg = model_vgg.fit(X_train, y_train,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            verbose=1,
                            callbacks=callbacks_vgg)

history_resnet = model_resnet.fit(X_train, y_train,
                                  epochs=epochs,
                                  validation_data=(X_val, y_val),
                                  verbose=1,
                                  callbacks=callbacks_resnet)


from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

for model_name in ["resnet", "vgg"]:
    best_model = keras.models.load_model(version+"/"+model_name+"/best_model.h5")
    y_pred = best_model.predict(X_test)
    y_pred_single_label = np.argmax(y_pred, axis=1)
    y_test_single_label = np.argmax(y_test, axis=1)
    precision = precision_score(y_test_single_label, y_pred_single_label, average='macro')
    recall = recall_score(y_test_single_label, y_pred_single_label, average='macro')
    f1 = f1_score(y_test_single_label, y_pred_single_label, average='macro')
    accuracy = accuracy_score(y_test_single_label, y_pred_single_label)
    precision = round(precision, 3)
    recall = round(recall, 3)
    f1 = round(f1, 3)
    accuracy = round(accuracy, 3)
    print(str(model_name))
    print(precision, recall, f1,
          accuracy)

Epoch 1/50
83/83 [==============================] - ETA: 0s - loss: 1.2373 - accuracy: 0.4415
Epoch 1: val_loss improved from inf to 1.10307, saving model to model_adding_ver8/vgg/best_model.h5
83/83 [==============================] - 27s 303ms/step - loss: 1.2373 - accuracy: 0.4415 - val_loss: 1.1031 - val_accuracy: 0.5884
Epoch 2/50
83/83 [==============================] - ETA: 0s - loss: 1.0271 - accuracy: 0.5611
Epoch 2: val_loss improved from 1.10307 to 0.94035, saving model to model_adding_ver8/vgg/best_model.h5
83/83 [==============================] - 22s 268ms/step - loss: 1.0271 - accuracy: 0.5611 - val_loss: 0.9404 - val_accuracy: 0.6769
Epoch 3/50
83/83 [==============================] - ETA: 0s - loss: 0.8862 - accuracy: 0.6440
Epoch 3: val_loss improved from 0.94035 to 0.81584, saving model to model_adding_ver8/vgg/best_model.h5
83/83 [==============================] - 22s 270ms/step - loss: 0.8862 - accuracy: 0.6440 - val_loss: 0.8158 - val_accuracy: 0.7211
Epoch 4/50
83/

Epoch 27/50
83/83 [==============================] - ETA: 0s - loss: 0.2038 - accuracy: 0.9440
Epoch 27: val_loss did not improve from 0.29984
83/83 [==============================] - 22s 268ms/step - loss: 0.2038 - accuracy: 0.9440 - val_loss: 0.3099 - val_accuracy: 0.8776
Epoch 28/50
83/83 [==============================] - ETA: 0s - loss: 0.2101 - accuracy: 0.9311
Epoch 28: val_loss improved from 0.29984 to 0.28025, saving model to model_adding_ver8/vgg/best_model.h5
83/83 [==============================] - 22s 271ms/step - loss: 0.2101 - accuracy: 0.9311 - val_loss: 0.2803 - val_accuracy: 0.8980
Epoch 29/50
83/83 [==============================] - ETA: 0s - loss: 0.1913 - accuracy: 0.9448
Epoch 29: val_loss improved from 0.28025 to 0.26778, saving model to model_adding_ver8/vgg/best_model.h5
83/83 [==============================] - 22s 270ms/step - loss: 0.1913 - accuracy: 0.9448 - val_loss: 0.2678 - val_accuracy: 0.9048
Epoch 30/50
83/83 [==============================] - ETA: 0s 

83/83 [==============================] - 16s 187ms/step - loss: 0.6539 - accuracy: 0.7491 - val_loss: 0.5158 - val_accuracy: 0.8095
Epoch 5/50
83/83 [==============================] - ETA: 0s - loss: 0.5610 - accuracy: 0.7915
Epoch 5: val_loss improved from 0.51576 to 0.50351, saving model to model_adding_ver8/resnet/best_model.h5
83/83 [==============================] - 16s 187ms/step - loss: 0.5610 - accuracy: 0.7915 - val_loss: 0.5035 - val_accuracy: 0.8265
Epoch 6/50
83/83 [==============================] - ETA: 0s - loss: 0.5200 - accuracy: 0.8089
Epoch 6: val_loss improved from 0.50351 to 0.49314, saving model to model_adding_ver8/resnet/best_model.h5
83/83 [==============================] - 16s 192ms/step - loss: 0.5200 - accuracy: 0.8089 - val_loss: 0.4931 - val_accuracy: 0.8129
Epoch 7/50
83/83 [==============================] - ETA: 0s - loss: 0.4741 - accuracy: 0.8188
Epoch 7: val_loss improved from 0.49314 to 0.43625, saving model to model_adding_ver8/resnet/best_model.h5
8

83/83 [==============================] - 15s 178ms/step - loss: 0.0653 - accuracy: 0.9864 - val_loss: 0.2251 - val_accuracy: 0.9184
Epoch 32/50
83/83 [==============================] - ETA: 0s - loss: 0.0674 - accuracy: 0.9856
Epoch 32: val_loss did not improve from 0.18255
83/83 [==============================] - 15s 180ms/step - loss: 0.0674 - accuracy: 0.9856 - val_loss: 0.2917 - val_accuracy: 0.9014
Epoch 33/50
83/83 [==============================] - ETA: 0s - loss: 0.0812 - accuracy: 0.9841
Epoch 33: val_loss did not improve from 0.18255
83/83 [==============================] - 15s 179ms/step - loss: 0.0812 - accuracy: 0.9841 - val_loss: 0.2224 - val_accuracy: 0.9252
Epoch 34/50
83/83 [==============================] - ETA: 0s - loss: 0.0715 - accuracy: 0.9807
Epoch 34: val_loss did not improve from 0.18255
83/83 [==============================] - 15s 180ms/step - loss: 0.0715 - accuracy: 0.9807 - val_loss: 0.1841 - val_accuracy: 0.9388
Epoch 35/50
83/83 [========================